# Abfragen zur Ausgabe relationaler Daten als JSON

[Format Query Results as JSON with FOR JSON](https:\learn.microsoft.com\en-us\sql\relational-databases\json\format-query-results-as-json-with-for-json-sql-server?view=sql-server-ver16)

## Lösung ohne Unterabfrage mit FOR JSON AUTO

Ergebnis: KundenBestellungen-Bsp1.json

In [ ]:
SELECT Kunde.KundenNr, Firma, Bestelldatum
    , CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS Rechnungsbetrag
FROM Kunde
JOIN Bestellung ON Kunde.KundenNr = Bestellung.KundenNr
    AND YEAR(Bestelldatum) = 2018
JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
WHERE Kunde.KundenNr <= 3
GROUP BY Kunde.KundenNr, Firma, Bestelldatum
ORDER BY Kunde.KundenNr, Bestelldatum
FOR JSON AUTO;

## Ausgewählte Kunden mit Rechnungen: Lösung mit korrelierter Unterabfrage und FOR JSON AUTO

Die korrelierte Unterabfrage gibt selbst mehrere Spalten im JSON Format aus.

Ergebnis: [KundenBestellungen-Bsp2.json]()

In [ ]:
SELECT Kunde.KundenNr, Kunde.Firma
    , (
        SELECT Bestelldatum, CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS Rechnungssumme
        FROM Bestellung 
        JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
        WHERE Kunde.KundenNr = Bestellung.KundenNr
        AND YEAR(Bestelldatum) = 2018
        GROUP BY Bestelldatum
        FOR JSON AUTO 
     ) AS Bestellungen
FROM Kunde 
WHERE KundenNr <= 3
FOR JSON AUTO;

## Ausgewählte Kunden mit Rechnungen und Details

Zur Steuerung der Verschachtelung werden hier sowohl FOR JSON AUTO als auch FOR JSON PATH eingesetzt.

- Äußere Abfrage zeigt Kunden
- Erste innere korrelierte Unterabfrage zeigt Rechnungsdatum und Summe
- Zweite innere Unterabfrage zeigt Rechnungspositionen

Ergebnis: [KundenBestellungen-Bsp3.json]()

In [ ]:
SELECT Kunde.KundenNr, Kunde.Firma
    , (
        SELECT Bestelldatum
            , CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS Rechnungssumme
            , (
               SELECT Produkt, Menge, CAST(Verkaufspreis AS DECIMAL(9,2)) AS Verkaufspreis
               FROM Produkt
               JOIN BestellDetail AS Details ON Produkt.ProduktID = Details.ProduktID
               AND Details.BestellungID = Bestellung.BestellungID 
               FOR JSON PATH
            ) AS Positionen
        FROM Bestellung 
        JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
        WHERE Kunde.KundenNr = Bestellung.KundenNr
        AND YEAR(Bestelldatum) = 2018
        GROUP BY Bestellung.BestellungID, Bestelldatum
        FOR JSON AUTO 
     ) AS Bestellungen
FROM Kunde 
WHERE KundenNr <= 3
FOR JSON AUTO;